In [2]:
import pandas as pd
import os
import boto3
from zipfile import ZipFile
import shutil
import sys
import os
sys.path.append(os.path.expanduser('../../'))
from scripts.utils.write_metadata import (
    append_metadata
)

In [31]:
# @append_metadata
def reproject_slr_census_blocks(varname='climate_pacific_institute_slr_exposure_susceptibility'):
    """
    This function pulls Pacific Institute sea level rise data from AWS. The csv file is modified for
    reprojection. The census block column was used to create an additional spatial column for census
    tracts as census blocks are at a higher spatial scale.
    
    Note:
    This function assumes users have configured the AWS CLI such that their access key / secret key pair are stored in ~/.aws/credentials.
    See https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html for guidance.
    """
    # Initialize the S3 client
    s3_client = boto3.client('s3')
    
    # Bucket name and file paths
    bucket_name = 'ca-climate-index'
    directory = '1_pull_data/climate_risk/sea_level_rise/exposure/projections/pacific_institute/vulnerable_slr_tract_2100.gdb.zip'
    out_directory = '2b_reproject/climate_risk/sea_level_rise/exposure/projections/pacific_institute/'
    
    # Local directory to store the downloaded zip file and extracted contents
    local_directory = 'temp'
    if not os.path.exists(local_directory):
        os.makedirs(local_directory)
    
    # Download the zip file
    print(f'Pulling vulnerable census blocks to SLR data from S3 bucket: {directory}')
    local_zip_file_path = os.path.join(local_directory, os.path.basename(directory))
    s3_client.download_file(bucket_name, directory, local_zip_file_path)
    
    # Extract the contents of the zip file
    with ZipFile(local_zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(local_directory)
    
    csv_found = False

    for filename in os.listdir(local_directory):
        if filename.endswith('.csv'):
            csv_path = os.path.join(local_directory, filename)
            print("Reading csv file")
            print('')
            slr_vulnerable_data = pd.read_csv(csv_path)
            print('.csv data pulled')
            print('')
            print('Original data:')
            print(slr_vulnerable_data)
            print('Adding Census Tract column to dataset')
            print('')
            slr_vulnerable_data['CensusTract'] = slr_vulnerable_data['CensusBlock'].astype(str).str[:10]
            print('Updated data:')
            print(slr_vulnerable_data)
            print('')
            print('Saving data as new .csv file and uploading to AWS')
            print('')
            
            # Save the modified DataFrame to a CSV file
            output_csv_path = os.path.join(local_directory, 'climate_pacific_institute_slr_exposure_census_vulnerability.csv')
            slr_vulnerable_data.to_csv(output_csv_path, index=False)
            print(f"Reprojected data saved to: {output_csv_path}")

            # Upload the CSV file to S3
            with open(output_csv_path, 'rb') as file:
                s3_client.upload_fileobj(file, bucket_name, os.path.join(out_directory, 'climate_pacific_institute_slr_exposure_census_vulnerability.csv'))

            print(f"Reprojected SLR data sent to S3 bucket: {out_directory}")
            break  # Break out of the loop after processing the first CSV file

    if not csv_found:
        print("No other CSV files found in the directory.")
                

In [30]:
reproject_slr_census_blocks(varname='climate_pacific_institute_slr_exposure_susceptibility')

Pulling vulnerable census blocks to SLR data from S3 bucket: 1_pull_data/climate_risk/sea_level_rise/exposure/projections/pacific_institute/vulnerable_slr_tract_2100.gdb.zip
Reading csv file

.csv data pulled

Original data:
         CensusBlock  CountyFIPS    BlkArea_m2  Perc_2000  Perc_2100
0     60014017001002           1  7.400145e+04   0.013629   0.103142
1     60014017001009           1  1.661440e+04   0.162421   0.818744
2     60014017001011           1  2.034830e+04   0.033993   0.145896
3     60014017002024           1  2.437019e+04   0.007894   0.054463
4     60014017002025           1  1.384990e+04   0.218054   0.323897
...              ...         ...           ...        ...        ...
9415  61110073001115         111  6.546174e+04   0.008397   0.037215
9416  61110073001120         111  2.331707e+04   0.010518   0.024219
9417  61110073001121         111  3.143619e+06   0.000482   0.001053
9418  61110073001124         111  2.357066e+04   0.006471   0.271358
9419  6111007300